# Ensemble Learning

## Imported Library for current project

In [14]:
import numpy as np

import pandas as pd

from pathlib import Path

from collections import Counter

from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import confusion_matrix

from imblearn.metrics import classification_report_imbalanced

import warnings

warnings.filterwarnings('ignore')

## Read the CSV and Perform Basic Data Cleaning

In [15]:
# Loaded the data

file_path = Path("Resources/LoanStats_2019Q1.csv")

df = pd.read_csv(file_path)

# Preview the data

df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [16]:
# Encoded features with LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(df["home_ownership"])

df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

le.fit(df["issue_d"])
df["issue_d"] = le.transform(df["issue_d"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

## Split the Data into Training and Testing

In [17]:
# Created features

x = df.copy()

x.drop("loan_status", axis=1, inplace=True)
# Created target

y = df['loan_status']

In [18]:
# Initialized Data

x.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [19]:
# Checked the balance of our target values

y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [20]:
# Splitted the x and y into x_train, x_test, y_train, y_test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [21]:
# Created the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [22]:
# Fitted the Standard Scaler with the training data 

x_scaler = scaler.fit(x_train)

In [23]:
# Scaled the training and testing data

x_train_scaled = x_scaler.transform(x_train)

x_test_scaled = x_scaler.transform(x_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [24]:
# Resampled the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

brf.fit(x_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [25]:
# Calculated the balanced accuracy score

y_pred = brf.predict(x_test)

balanced_accuracy_score(y_test, y_pred)

0.7290249400290825

In [26]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   62,    39],
       [ 2665, 14439]], dtype=int64)

In [27]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0227    0.6139    0.8442    0.0438    0.7199    0.5063       101
   low_risk     0.9973    0.8442    0.6139    0.9144    0.7199    0.5302     17104

avg / total     0.9916    0.8428    0.6152    0.9093    0.7199    0.5300     17205



In [28]:
# Listed the features sorted in descending order by feature importance

sorted(zip(brf.feature_importances_, x.columns), reverse=True)

[(0.07646711017201446, 'total_rec_prncp'),
 (0.0681731200022894, 'last_pymnt_amnt'),
 (0.05699252404641979, 'total_pymnt'),
 (0.05424778840861491, 'total_pymnt_inv'),
 (0.04735017423439608, 'total_rec_int'),
 (0.031245651401774252, 'int_rate'),
 (0.022619745828795626, 'issue_d'),
 (0.02244519942392171, 'mths_since_recent_inq'),
 (0.020409215004023765, 'installment'),
 (0.01808337089191728, 'tot_hi_cred_lim'),
 (0.017004195413593538, 'bc_util'),
 (0.01640559494238341, 'dti'),
 (0.016118934293676203, 'mths_since_rcnt_il'),
 (0.01581716497633785, 'il_util'),
 (0.015630310867867408, 'max_bal_bc'),
 (0.015494921031886601, 'out_prncp'),
 (0.015281199871082264, 'out_prncp_inv'),
 (0.01470961095035287, 'mo_sin_old_il_acct'),
 (0.014693851231416522, 'mo_sin_old_rev_tl_op'),
 (0.01425370693271944, 'total_bal_ex_mort'),
 (0.014094010852862895, 'total_bc_limit'),
 (0.013976327240119397, 'revol_bal'),
 (0.013793435235420952, 'avg_cur_bal'),
 (0.013416065136567646, 'tot_cur_bal'),
 (0.01339859137918

### Easy Ensemble Classifier

In [29]:
# Train the Classifier

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

eec.fit(x_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [30]:
# Calculated the balanced accuracy score

y_pred1 = eec.predict(x_test)

balanced_accuracy_score(y_test, y_pred1)

0.9316600714093861

In [31]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred1)

array([[   93,     8],
       [  983, 16121]], dtype=int64)

In [32]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred1, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.0864    0.9208    0.9425    0.1580    0.9316    0.8660       101
   low_risk     0.9995    0.9425    0.9208    0.9702    0.9316    0.8698     17104

avg / total     0.9941    0.9424    0.9209    0.9654    0.9316    0.8697     17205



## Final Questions

Q1. Which model had the best balanced accuracy score?

    A: The Easy Ensemble Classifier has the best balanced accuracy score at 93.17% 

Q2. Which model had the best recall score?

    A: The Easy Ensemble Classifier has the best recall score at 94.24%

Q3. Which model had the best geometric mean score?

    A: The Easy Ensemble Classifier has the best geometric mean score at 93.16%.

Q4. What are the top three features?


    A: The top three features initialized by the Balanced Random Forest Classifier;
    
        i. Total Recall Principal (7.6%)
   
        ii. Last Payment Amount (6.8%)

        iii. Total Payment (5.7%)